# Spark Structured Streaming Example in DSX
This code demonstrate how to run Structured Streaming quick example from Apache Spark Documentation.

Let’s say we needto maintain a running word count of text data received from a data server listening on a TCP socket. Keep this data in an in-memory tablem, so that we can query using Spark SQL.

- Read Spark's Structured Streaming programing Guide here https://spark.apache.org/docs/2.1.0/structured-streaming-programming-guide.html
- Express the required logic for word count using structured streaming api
- Lets use netcat utility on unix as a data server to test it.

### > Create a DataFrame representing the stream of input lines from connection to localhost:9999
 <div class="panel-group" id="accordion-11">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-11" href="#collapse1-11">
        Hint</a>
      </h4>
    </div>
    <div id="collapse1-11" class="panel-collapse collapse">
      <div class="panel-body">Use <i>readStream</i> method on the spark session with socket source format and other options</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-11" href="#collapse2-11">
        Show Code</a>
      </h4>
    </div>
    <div id="collapse2-11" class="panel-collapse collapse">
      <div class="panel-body">
lines&nbsp;=&nbsp;spark&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.readStream&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.format("socket")&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.option("host",&nbsp;"localhost")&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.option("port",&nbsp;9999)&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.load()</div>
    </div>
  </div>
</div> 

### > Split each line into words
 <div class="panel-group" id="accordion-22">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-22" href="#collapse1-22">
        Hint</a>
      </h4>
    </div>
    <div id="collapse1-22" class="panel-collapse collapse">
      <div class="panel-body"> Use pyspark's built-in SQL functions - <i>split</i> and <i>explode</i>, to split each line into multiple rows with a word each to define <i>words</i> dataframe.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-22" href="#collapse2-22">
        Show Code</a>
      </h4>
    </div>
    <div id="collapse2-22" class="panel-collapse collapse">
      <div class="panel-body">
#Import&nbsp;required&nbsp;pyspark&nbsp;sql&nbsp;unctions<br>
from&nbsp;pyspark.sql.functions&nbsp;import&nbsp;explode<br>
from&nbsp;pyspark.sql.functions&nbsp;import&nbsp;split<br>
<br>
words&nbsp;=&nbsp;lines.select(<br>
&nbsp;&nbsp;&nbsp;explode(<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;split(lines.value,&nbsp;"&nbsp;")<br>
&nbsp;&nbsp;&nbsp;).alias("word")<br>
)</div>
    </div>
  </div>
</div> 

### > Generate running word count
 <div class="panel-group" id="accordion-33">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-33" href="#collapse1-33">
        Hint</a>
      </h4>
    </div>
    <div id="collapse1-33" class="panel-collapse collapse">
      <div class="panel-body">Define the wordCounts DataFrame by grouping by the unique values in the Dataset and counting them.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-22" href="#collapse2-33">
        Show Code</a>
      </h4>
    </div>
    <div id="collapse2-33" class="panel-collapse collapse">
      <div class="panel-body">
wordCounts&nbsp;=&nbsp;words.groupBy("word").count()
        </div>
    </div>
  </div>
</div> 

### > Setup a query on wordCount steaming dataframe.
- Setup a query for complete set of counts (specified by outputMode("complete")) to a in-memory table with name TB_WORD_COUNT every time they are updated.
- Set query trigger frequency to 30 seconds
 <div class="panel-group" id="accordion-44">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-33" href="#collapse1-44">
        Hint</a>
      </h4>
    </div>
    <div id="collapse1-44" class="panel-collapse collapse">
      <div class="panel-body">Use dataframe's writeStream function with outputMode as "complete" to compute complete set of counts<br> Use output format for in-memory table and trigger for tigger frequency.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-22" href="#collapse2-44">
        Show Code</a>
      </h4>
    </div>
    <div id="collapse2-44" class="panel-collapse collapse">
      <div class="panel-body">
query&nbsp;=&nbsp;wordCounts&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.writeStream&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.outputMode("complete")&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.trigger(processingTime="30&nbsp;seconds")&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.format("memory")&nbsp;\<br>
&nbsp;&nbsp;&nbsp;&nbsp;.queryName("TB_WORD_COUNT")<br>
        </div>
    </div>
  </div>
</div> 

## Before starting the query start a data server for testing

### > Run netcat command and output a line every 15 seconds
      Run it in separate thread, so that we can execute other cells

In [ ]:
%%script bash --bg
while sleep 5; do echo "Spark is fast. DSX and Spark combination is awesome"; done | nc -l 9999

### > Now start the query  to build the output table

In [ ]:
rquery =  query.start()

### > Query the table to see the results
- Execute below query multiple times to see the change of values for running counts.

In [ ]:
spark.sql("select * from TB_WORD_COUNT").show()

### > Finally Stop the streaming query 

In [ ]:
rquery.stop()

### > Stop kernal to stop netcat server in backhround thread.